In [ ]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  # Mount the Google Drive to access files stored there
  drive.mount('/content/drive')

  # Install the latest version of torchtext library quietly without showing output

  !pip install transformers evaluate wandb datasets accelerate  -U -qq  ## NEW LINES ##

  basepath = '/content/drive/MyDrive/NLP/'

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.0 MB/s eta 0:00:00


In [ ]:
# Importing PyTorch library for tensor computations and neural network modules
import torch
import torch.nn as nn

# For working with textual data vocabularies and for displaying model summaries

# General-purpose Python libraries for random number generation and numerical operations
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Utilities for efficient serialization/deserialization of Python objects and for element tallying
import joblib
from collections import Counter

# For creating lightweight attribute classes and for partial function application
from functools import partial

# For filesystem path handling, generating and displaying confusion matrices, and date-time manipulations
from pathlib import Path
from sklearn.metrics import confusion_matrix
from datetime import datetime

# For plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

### NEW ##########################
# imports from Huggingface ecosystem
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import evaluate

# wandb library
import wandb

from sklearn.base import BaseEstimator, TransformerMixin
from bs4 import BeautifulSoup
import re
import spacy
import numpy as np
from nltk.stem.porter import PorterStemmer
import os


In [ ]:
base_folder = Path(basepath)
data_folder = base_folder/'Data Folder'
model_folder = base_folder/'Models'
custom_functions = base_folder/'custom-functions'

In [ ]:
train_df = pd.read_csv(data_folder/'train.csv')
#test_df = pd.read_csv(data_folder/'test.csv')

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 6.8 MB/s eta 0:00:00


In [ ]:
import re
import emoji
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove special characters and numbers
    text = re.sub(r'\W+|\d+', ' ', text)
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    text = emoji.demojize(text)
    # Convert to lowercase
    text = text.lower()
    return text

In [ ]:
train_df['Tweet'] = train_df['Tweet'].apply(preprocess_text)

In [ ]:
# Separate features and target labels
X = train_df['Tweet']  # Features (the Tweet text)
y = train_df.drop(columns=['ID', 'Tweet'])  # Labels (emotion classes)

# Count occurrences of each class
class_counts = y.sum(axis=0)  # Count the number of positive samples for each label
print("Class counts:\n", class_counts)

# Identify labels with less than 2 samples
labels_to_remove = class_counts[class_counts < 2].index.tolist()

# Drop these labels from the DataFrame
y_reduced = y.drop(columns=labels_to_remove)

# Re-create binary matrix for stratification
y_binary_reduced = y_reduced.values  # Convert to numpy array

# Perform the train-test split without stratification
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y_binary_reduced,
    test_size=0.2,
    random_state=42,
)

# Convert y_train and y_val back to DataFrame if necessary
y_train_df = pd.DataFrame(y_train, columns=y_reduced.columns)
y_val_df = pd.DataFrame(y_val, columns=y_reduced.columns)

# Create Hugging Face datasets
y_train = y_train_df.values.tolist()  # Convert to list of lists
y_valid = y_val_df.values.tolist()  # Convert to list of lists

trainset = Dataset.from_dict({
    'texts': X_train.tolist(),  # Convert Series to list
    'labels': y_train           # Your multilabels for training
})

validset = Dataset.from_dict({
    'texts': X_val.tolist(),  # Convert Series to list
    'labels': y_valid           # Your multilabels for validation
})

# Print the dataset sizes
print("Training dataset size:", len(trainset))
print("Validation dataset size:", len(validset))

Class counts:
 anger           2859
anticipation    1102
disgust         2921
fear            1363
joy             2877
love             832
optimism        2291
pessimism        895
sadness         2273
surprise         396
trust            400
dtype: int64
Training dataset size: 6179
Validation dataset size: 1545


In [ ]:
print(trainset)

Dataset({
    features: ['texts', 'labels'],
    num_rows: 6179
})


In [ ]:
trainset[0]

{'texts': 'going to get myself a copy of s cujo for an upcoming project that i can t talk about just yet',
 'labels': [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0]}

In [ ]:
from transformers import PretrainedConfig

class CustomConfig(PretrainedConfig):
    def __init__(self, vocab_size=0, embedding_dim=0, hidden_dim1=0, hidden_dim2=0, num_labels=11, **kwargs):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2
        self.num_labels = num_labels

In [ ]:
import torch
import torch.nn as nn
from transformers import PreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput

class CustomMLP(PreTrainedModel):
    config_class = CustomConfig

    def __init__(self, config):
        super().__init__(config)

        self.embedding_bag = nn.EmbeddingBag(config.vocab_size, config.embedding_dim)
        self.layers = nn.Sequential(
            nn.Linear(config.embedding_dim, config.hidden_dim1),
            nn.BatchNorm1d(num_features=config.hidden_dim1),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(config.hidden_dim1, config.hidden_dim2),
            nn.BatchNorm1d(num_features=config.hidden_dim2),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(config.hidden_dim2, 11)  # Output layer
        )
    def forward(self, input_ids, offsets, labels=None):
      embed_out = self.embedding_bag(input_ids, offsets)
      logits = self.layers(embed_out)
      loss = None
      if labels is not None:
        if labels.dim() == 1:
          labels = labels.unsqueeze(1)

        # Print shapes for debugging
        #print(f"logits shape: {logits.shape}, labels shape: {labels.shape}")

        # Ensure labels are float
        labels = labels.float()

        # Calculate the loss using BCEWithLogitsLoss
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels)  # logits already include sigmoid

      return SequenceClassifierOutput(
        loss=loss,
        logits=logits
    )



In [ ]:
from collections import Counter, OrderedDict
from typing import Dict, List, Optional, Union

class Vocab:
    def __init__(self, tokens: List[str]) -> None:
        self.itos: List[str] = tokens
        self.stoi: Dict[str, int] = {token: i for i, token in enumerate(tokens)}
        self.default_index: Optional[int] = None

    def __getitem__(self, token: str) -> int:
        if token in self.stoi:
            return self.stoi[token]
        if self.default_index is not None:
            return self.default_index
        raise RuntimeError(f"Token '{token}' not found in vocab")

    def __contains__(self, token: str) -> bool:
        return token in self.stoi

    def __len__(self) -> int:
        return len(self.itos)

    def insert_token(self, token: str, index: int) -> None:
        if index < 0 or index > len(self.itos):
            raise ValueError("Index out of range")
        if token in self.stoi:
            old_index = self.stoi[token]
            if old_index < index:
                self.itos.pop(old_index)
                self.itos.insert(index - 1, token)
            else:
                self.itos.pop(old_index)
                self.itos.insert(index, token)
        else:
            self.itos.insert(index, token)

        self.stoi = {token: i for i, token in enumerate(self.itos)}

    def append_token(self, token: str) -> None:
        if token in self.stoi:
            raise RuntimeError(f"Token '{token}' already exists in the vocab")
        self.insert_token(token, len(self.itos))

    def set_default_index(self, index: Optional[int]) -> None:
        self.default_index = index

    def get_default_index(self) -> Optional[int]:
        return self.default_index

    def lookup_token(self, index: int) -> str:
        if 0 <= index < len(self.itos):
            return self.itos[index]
        raise RuntimeError(f"Index {index} out of range")

    def lookup_tokens(self, indices: List[int]) -> List[str]:
        return [self.lookup_token(index) for index in indices]

    def lookup_indices(self, tokens: List[str]) -> List[int]:
        return [self[token] for token in tokens]

    def get_stoi(self) -> Dict[str, int]:
        return self.stoi.copy()

    def get_itos(self) -> List[str]:
        return self.itos.copy()

    @classmethod
    def vocab(cls, ordered_dict: Union[OrderedDict, Counter], min_freq: int = 1, specials: Optional[List[str]] = None, special_first: bool = True) -> 'Vocab':
        specials = specials or []
        for token in specials:
            ordered_dict.pop(token, None)

        tokens = [token for token, freq in ordered_dict.items() if freq >= min_freq]

        if special_first:
            tokens = specials + tokens
        else:
            tokens = tokens + specials

        return cls(tokens)

In [ ]:
def get_vocab(dataset, min_freq=1):
    """
    Generate a vocabulary from a dataset.

    Args:
        dataset (Dataset): A Hugging Face Dataset object. The dataset should
                           have a key 'texts' that contains the text data.
        min_freq (int): The minimum frequency for a token to be included in
                        the vocabulary.

    Returns:
        torchtext.vocab.Vocab: Vocabulary object containing tokens from the
                               dataset that meet or exceed the specified
                               minimum frequency. It also includes a special
                               '<unk>' token for unknown words.
    """
    # Initialize a counter object to hold token frequencies
    counter = Counter()

    # Update the counter with tokens from each text in the dataset
    # Iterating through texts in the dataset
    for text in dataset['texts']:  ###### Change from previous function ####
        counter.update(str(text).split())

    # Create a vocabulary using the counter object
    # Tokens that appear fewer times than `min_freq` are excluded
    my_vocab = Vocab.vocab(counter, min_freq=min_freq)

    # Insert a '<unk>' token at index 0 to represent unknown words
    my_vocab.insert_token('<unk>', 0)

    # Set the default index to 0
    # This ensures that any unknown word will be mapped to '<unk>'
    my_vocab.set_default_index(0)

    return my_vocab

In [ ]:
# Creating a function that will be used to get the indices of words from vocab
def tokenizer(text, vocab):
    """Converts text to a list of indices using a vocabulary dictionary"""
    return [vocab[token] for token in str(text).split()]

In [ ]:
def collate_batch(batch, my_vocab):
  labels = [sample['labels'] for sample in batch]
  texts = [sample['texts'] for sample in batch]

  labels = torch.tensor(labels, dtype =torch.float32) #check this later

  list_of_list_of_indices = [tokenizer(text, my_vocab) for text in texts]
 #check this later
    # Concatenate all text indices into a single tensor
  input_ids = torch.cat([torch.tensor(i, dtype=torch.int64) for i in list_of_list_of_indices])

    # Compute the offsets for each text in the concatenated tensor
  offsets = [0] + [len(i) for i in list_of_list_of_indices]
  offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)

  return {
        'input_ids': input_ids,
        'offsets': offsets,
        'labels': labels
  }

In [ ]:
multi_label_vocab = get_vocab(trainset, min_freq=2)
collate_fn = partial(collate_batch, my_vocab=multi_label_vocab)

In [ ]:
my_config = CustomConfig(vocab_size=len(multi_label_vocab),
                         embedding_dim=300,
                         hidden_dim1=200,
                         hidden_dim2=100,
                         num_labels=11)  # Update to the total number of unique labels


In [ ]:
my_config.id2label = {
    0: 'anger',
    1: 'anticipation',
    2: 'disgust',
    3: 'fear',
    4: 'joy',
    5: 'love',
    6: 'optimism',
    7: 'pessimism',
    8: 'sadness',
    9: 'surprise',
    10: 'trust'
}


In [ ]:
# Generating id_to_label by reversing the key-value pairs in label_to_id
my_config.label2id = {v: k for k, v in my_config.id2label .items()}

In [ ]:
my_config

CustomConfig {
  "embedding_dim": 300,
  "hidden_dim1": 200,
  "hidden_dim2": 100,
  "id2label": {
    "0": "anger",
    "1": "anticipation",
    "2": "disgust",
    "3": "fear",
    "4": "joy",
    "5": "love",
    "6": "optimism",
    "7": "pessimism",
    "8": "sadness",
    "9": "surprise",
    "10": "trust"
  },
  "label2id": {
    "anger": 0,
    "anticipation": 1,
    "disgust": 2,
    "fear": 3,
    "joy": 4,
    "love": 5,
    "optimism": 6,
    "pessimism": 7,
    "sadness": 8,
    "surprise": 9,
    "trust": 10
  },
  "transformers_version": "4.46.1",
  "vocab_size": 4974
}

In [ ]:
my_config

CustomConfig {
  "embedding_dim": 300,
  "hidden_dim1": 200,
  "hidden_dim2": 100,
  "id2label": {
    "0": "anger",
    "1": "anticipation",
    "2": "disgust",
    "3": "fear",
    "4": "joy",
    "5": "love",
    "6": "optimism",
    "7": "pessimism",
    "8": "sadness",
    "9": "surprise",
    "10": "trust"
  },
  "label2id": {
    "anger": 0,
    "anticipation": 1,
    "disgust": 2,
    "fear": 3,
    "joy": 4,
    "love": 5,
    "optimism": 6,
    "pessimism": 7,
    "sadness": 8,
    "surprise": 9,
    "trust": 10
  },
  "transformers_version": "4.46.1",
  "vocab_size": 4974
}

In [ ]:
model = CustomMLP(config=my_config)


In [ ]:
model

CustomMLP(
  (embedding_bag): EmbeddingBag(4974, 300, mode='mean')
  (layers): Sequential(
    (0): Linear(in_features=300, out_features=200, bias=True)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=100, out_features=11, bias=True)
  )
)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, classification_report
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    sigmoid_logits = torch.sigmoid(torch.tensor(logits))
    predictions = (sigmoid_logits >= 0.5).int().numpy()
    labels = labels.astype(int)

    f1_micro = f1_score(labels, predictions, average='micro', zero_division=0)
    f1_macro = f1_score(labels, predictions, average='macro', zero_division=0)

    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
    }

In [ ]:
# Configure training parameters
training_args = TrainingArguments(
    num_train_epochs=100,
    per_device_train_batch_size=118,  # Adjust if necessary
    per_device_eval_batch_size=118,    # Adjust if necessary
    weight_decay=0.1,
    learning_rate=0.001,
    optim='adamw_torch',
    remove_unused_columns=False,

    output_dir=str(model_folder),
    evaluation_strategy='steps',
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    load_best_model_at_end=True,
    save_total_limit=2,

    # Set metric_for_best_model to a multilabel metric like "f1" if computed in `compute_metrics`
    metric_for_best_model="eval_f1_micro",  # Use F1 score for multilabel classification
    greater_is_better=True,

    logging_strategy='steps',
    logging_steps=200,
    report_to='wandb',
    run_name='multi_hf_trainer',
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset=validset,
    data_collator=collate_fn,  # Make sure this function is correctly handling multilabel data
    compute_metrics=compute_metrics,  # Ensure this function computes metrics suitable for multilabel tasks
)


In [ ]:
trainer.train()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,F1 Micro,F1 Macro
200,0.491800,0.454287,0.313749,0.166824
400,0.402600,0.433639,0.453366,0.270831
600,0.355200,0.429531,0.481308,0.305757
800,0.327700,0.429864,0.489209,0.314273
1000,0.307100,0.439505,0.495053,0.325961
1200,0.289400,0.450482,0.506776,0.341949
1400,0.276600,0.456034,0.505488,0.348714
1600,0.264100,0.459454,0.517037,0.365633
1800,0.252400,0.472404,0.521926,0.368815
2000,0.244200,0.474830,0.516910,0.370831


TrainOutput(global_step=5300, training_loss=0.24436214734923165, metrics={'train_runtime': 290.1226, 'train_samples_per_second': 2129.789, 'train_steps_per_second': 18.268, 'total_flos': 85190632160400.0, 'train_loss': 0.24436214734923165, 'epoch': 100.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.5137145519256592,
 'eval_f1_micro': 0.5253956292388847,
 'eval_f1_macro': 0.38546617838143077,
 'eval_runtime': 0.1245,
 'eval_samples_per_second': 12414.461,
 'eval_steps_per_second': 112.493,
 'epoch': 100.0}

In [ ]:
valid_output = trainer.predict(validset)

In [ ]:
valid_output._fields

('predictions', 'label_ids', 'metrics')

In [ ]:
import numpy as np

# Assuming valid_output.predictions contains logits from your model
# Apply a sigmoid activation to convert logits to probabilities
valid_probs = valid_output.predictions  # These should be raw logits
valid_preds = (valid_probs > 0.5).astype(int)  # Convert to binary predictions based on threshold
valid_labels = np.array(valid_output.label_ids).astype(int)


In [ ]:
valid_output.metrics

{'test_loss': 0.5137145519256592,
 'test_f1_micro': 0.5253956292388847,
 'test_f1_macro': 0.38546617838143077,
 'test_runtime': 0.1736,
 'test_samples_per_second': 8900.884,
 'test_steps_per_second': 80.655}

In [ ]:
# After training, let us check the best checkpoint
# We need this for Inference
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")

The best model was saved at step 2800.


In [ ]:
wandb.finish()

eval/f1_macro,▁▄▅▆▆▆▇▇▇▇▇████████████████
eval/f1_micro,▁▆▇▇▇▇▇████████████████████
eval/loss,▂▁▁▁▁▂▂▂▃▃▃▄▄▅▅▆▆▇▇▇▇█████▅
eval/runtime,▇▂▆▃▅▂▅▁▂▂▂█▁▅▂▂▂▁█▂▇▄▆▂▃▁▁
eval/samples_per_second,▂▇▂▆▃▇▄█▆▇▇▁▇▃▇▇▇█▁▆▂▅▂▆▅▇█
eval/steps_per_second,▂▇▂▆▃▇▄█▆▇▇▁▇▃▇▇▇█▁▆▂▅▂▆▅▇█
test/f1_macro,▁
test/f1_micro,▁
test/loss,▁
test/runtime,▁
test/samples_per_second,▁


In [ ]:
test_data=pd.read_csv(data_folder/'test.csv')

In [ ]:
test_data.head()

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-01559,@Adnan__786__ @AsYouNotWish Dont worry Indian ...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
1,2018-03739,"Academy of Sciences, eschews the normally sobe...",NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
2,2018-00385,I blew that opportunity -__- #mad,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
3,2018-03001,This time in 2 weeks I will be 30... 😥,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
4,2018-01988,#Deppression is real. Partners w/ #depressed p...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE


In [ ]:
test_data['Tweet'] = test_data['Tweet'].apply(preprocess_text)

In [ ]:
X_test = test_data['Tweet'].values

In [ ]:
device = 'cpu'
# Convert the list of texts into a list of lists; each inner list contains the vocabulary indices for a text
list_of_list_of_indices = [tokenizer(text, multi_label_vocab) for text in X_test]

# Compute the offsets for each text in the concatenated tensor
offsets = [0] + [len(i) for i in list_of_list_of_indices]
offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)

# Concatenate all text indices into a single tensor
indices = torch.cat([torch.tensor(i, dtype=torch.int64) for i in list_of_list_of_indices])
indices

# move model to appropriate device
device = 'cpu'
model.to(device)

# put model in evaluation mode
model.eval()

# get outputs (logits) from model
outputs = model(indices, offsets)



In [ ]:
# get predicted labels
probabilities = torch.sigmoid(outputs.logits)
threshold = 0.5
predicted_labels = (probabilities >= threshold).int()
no_labels = predicted_labels.sum(1) == 0
predicted_labels[no_labels, torch.max(probabilities[no_labels], dim=1).indices] = 1

In [ ]:
binary_predictions = predicted_labels
predicted_label_names = []
for prediction in binary_predictions:
    labels = [model.config.id2label[i] for i, label in enumerate(prediction) if label == 1]
    predicted_label_names.append(labels)

In [ ]:
result_df = pd.DataFrame(predicted_labels.numpy(), columns=my_config.id2label.values())
result_df.insert(0, 'ID', test_data['ID'])
result_df


,ID,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-01559,1,0,1,1,0,0,0,0,0,0,0
1,2018-03739,1,0,1,0,0,0,0,0,1,0,0
2,2018-00385,0,0,0,0,0,0,1,0,1,0,0
3,2018-03001,0,0,0,0,1,0,1,0,0,0,0
4,2018-01988,1,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3254,2018-03848,0,0,1,0,0,0,0,0,0,0,0
3255,2018-00416,1,0,1,1,0,0,0,0,0,0,0
3256,2018-03717,1,0,1,0,0,0,0,0,0,0,0
3257,2018-03504,0,0,0,0,1,1,1,0,0,0,0


In [ ]:
result_df.to_csv(data_folder/'submission.csv', index=False)